In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from scipy import stats
import statsmodels.api as sm

from sklearn.model_selection import GridSearchCV,TimeSeriesSplit, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, make_scorer
from sklearn.metrics import recall_score, precision_score, accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

from collections import Counter
from scipy.stats import loguniform, uniform as stats_uniform

# Scikit-learn e Imblearn
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, f1_score, accuracy_score, precision_score, recall_score, roc_auc_score

# Imblearn
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# carregando dataset
train =pd.read_csv("/content/drive/MyDrive/MESTRADO/CMP Aprendizado de Máquina/TRABALHO/DADOS ESSE/train_data (1).csv")
val =pd.read_csv("/content/drive/MyDrive/MESTRADO/CMP Aprendizado de Máquina/TRABALHO/DADOS ESSE/val_data (1).csv")
test = pd.read_csv("/content/drive/MyDrive/MESTRADO/CMP Aprendizado de Máquina/TRABALHO/DADOS ESSE/test_data (1).csv")

**REGRESSAO LOGISTICA CROSS VALIDATION VANILA**

In [12]:
# dividindo em treino e teste para treinamento do modelo
X_train = train.drop(['patient_id','date','label'], axis=1)
y_train = train['label']

X_val = val.drop(['patient_id','date','label'], axis=1)
y_val = val['label']

X_test = test.drop(['patient_id','date','label'], axis=1)
y_test = test['label']

In [13]:
import numpy as np
from sklearn.metrics import accuracy_score # Necessário para a avaliação final

# A função assume que X_train, y_train, pipeline, outer_tscv, inner_tscv,
# param_distributions, scoring, n_iter e RANDOM_STATE estão definidos no escopo global.

def perform_nested_cv_corrected_accuracy(X_train, y_train, pipeline, inner_tscv, outer_tscv, param_distributions, scoring, n_iter, RANDOM_STATE):

    # Listas para armazenar os resultados (mudança de nome para refletir 'accuracy')
    nested_scores_accuracy = []
    nested_best_params = []
    nested_best_scores_inner = []

    # 1. Loop Externo
    for i, (outer_train_index, outer_test_index) in enumerate(outer_tscv.split(X_train, y_train)):
        print(f"\n--- Outer Split {i+1}/{outer_tscv.get_n_splits()} ---")

        # Split de dados (ORIGINAIS)
        # Adicionei .copy() para boas práticas, embora .iloc funcione bem
        X_train_outer, X_test_outer = X_train.iloc[outer_train_index].copy(), X_train.iloc[outer_test_index].copy()
        y_train_outer, y_test_outer = y_train.iloc[outer_train_index].copy(), y_train.iloc[outer_test_index].copy()

        print(f"Outer Train Data Shape (Original): {X_train_outer.shape}")
        print(f"Outer Test Data Shape (Original): {X_test_outer.shape}")

        # 2. Loop Interno (RandomizedSearchCV)
        print("Starting inner Randomized Search (SMOTETomek in Pipeline)...")

        inner_search = RandomizedSearchCV(
            estimator=pipeline, # Usa o ImbPipeline com SMOTETomek
            param_distributions=param_distributions,
            n_iter=n_iter,
            cv=inner_tscv, # Validador interno
            scoring=scoring,
            refit='accuracy', # 🎯 CORREÇÃO 1: O modelo será reajustado com base na 'accuracy'
            random_state=RANDOM_STATE,
            n_jobs=-1,
            return_train_score=True,
            error_score=0.0
        )

        # FIT nos dados de treino externos ORIGINAIS (ImbPipeline faz a amostragem internamente)
        inner_search.fit(X_train_outer, y_train_outer)

        print("Inner Search completed.")

        # Armazenar os resultados internos (accuracy)
        nested_best_params.append(inner_search.best_params_)
        nested_best_scores_inner.append(inner_search.best_score_)
        print(f"Inner CV best params (refit='accuracy'): {inner_search.best_params_}")
        print(f"Inner CV best accuracy score: {inner_search.best_score_}") # Print corrigido

        # 3. Avaliação no Teste Externo (Métrica final)
        print("Evaluating best model on the outer test fold...")
        best_inner_model = inner_search.best_estimator_

        # O modelo treinado já é o ImbPipeline
        y_pred_outer = best_inner_model.predict(X_test_outer)

        # 🎯 CORREÇÃO 2: Usamos accuracy_score como métrica de desempenho final
        outer_accuracy_score = accuracy_score(y_test_outer, y_pred_outer)

        nested_scores_accuracy.append(outer_accuracy_score)
        print(f"Outer fold Accuracy score: {outer_accuracy_score}") # Print corrigido
        print("-" * 50)

    # 4. Resultados Finais
    print("\n--- Resultados Finais da Validação Cruzada Aninhada ---")
    print(f"Nested CV Accuracy scores (por fold): {nested_scores_accuracy}") # Print corrigido
    print(f"Mean Nested CV Accuracy score: {np.mean(nested_scores_accuracy)}") # Print corrigido
    print(f"Standard deviation of Nested CV Accuracy scores: {np.std(nested_scores_accuracy)}") # Print corrigido

    # Retorno corrigido
    return nested_scores_accuracy, nested_best_params, nested_best_scores_inner, np.std(nested_scores_accuracy)

In [14]:
def evaluate(model, X, y, set_name="set"):
    """Avalia o modelo em um conjunto de dados e exibe métricas."""
    y_pred = model.predict(X)

    # Verifica se o modelo tem predict_proba antes de chamar
    if hasattr(model, 'predict_proba'):
        y_proba = model.predict_proba(X)[:, 1]
    else:
        y_proba = None

    acc = accuracy_score(y, y_pred)
    # Define zero_division=0 para evitar avisos e garantir que o relatório seja calculado
    report = classification_report(y, y_pred, digits=4, zero_division=0)
    cm = confusion_matrix(y, y_pred)

    print(f"\n-------------------------------------------- Avaliação no {set_name} -------------------------------------------\n")
    print("Accuracy:", acc)
    print("\nClassification report:\n", report)
    print("Confusion matrix:\n", cm)

    # Plota Matriz de Confusão
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(f"Matriz de Confusão - {set_name}")
    plt.xlabel("Previsto")
    plt.ylabel("Verdadeiro")
    plt.show()

    unique_classes = np.unique(y)
    if len(unique_classes) == 2 and y_proba is not None:
        auc = roc_auc_score(y, y_proba)
        print("\nROC AUC:", auc)
        fpr, tpr, thr = roc_curve(y, y_proba)

        # Plota Curva ROC
        plt.figure(figsize=(6, 4))
        plt.plot(fpr, tpr, label=f"AUC = {auc:.4f}")
        plt.plot([0,1],[0,1],"--")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title(f"Curva ROC - {set_name}")
        plt.legend(loc="lower right")
        plt.grid(True)
        plt.show()

    return {"accuracy": acc, "report_str": report, "confusion_matrix": cm}


In [15]:
# --- PARÂMETROS GLOBAIS ---
RANDOM_STATE = 42
n_iter = 50
n_splits_outer = 5 # Para o TimeSeriesSplit externo
n_splits_inner = 5 # Para o TimeSeriesSplit interno

# Definindo os validadores
outer_tscv = TimeSeriesSplit(n_splits=n_splits_outer)
inner_tscv = TimeSeriesSplit(n_splits=n_splits_inner)

# --- DEFINIÇÃO DO PIPELINE COM SMOTETOMEK ---
model_base = LogisticRegression(random_state=RANDOM_STATE, n_jobs=-1)

smote_tomek_instance = SMOTETomek(
    smote=SMOTE(random_state=RANDOM_STATE),
    tomek=TomekLinks(n_jobs=-1), # Parallelismo para undersampling
    random_state=RANDOM_STATE
)

# Pipeline CORRETO: SMOTETomek + Classificador (Usando ImbPipeline)
pipeline = ImbPipeline(steps=[
    ('smote_tomek', smote_tomek_instance), # Passo de amostragem
    ('model', model_base)                  # Passo do modelo (LogisticRegression)
])

# --- METRICAS DE SCORING ---
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, zero_division=0),
    'recall': make_scorer(recall_score, zero_division=0),
    'f1': make_scorer(f1_score, zero_division=0), # Métrica de refit
    'roc_auc': make_scorer(roc_auc_score, zero_division=0)
}

# --- PARAM_DISTRIBUTIONS CORRIGIDO E OTIMIZADO ---
param_distributions = [
    # l1 penalty (Solver: liblinear)
    {'model__penalty': ['l1'], 'model__solver': ['liblinear'], 'model__C': loguniform(0.0001, 100.0), 'model__max_iter': [10000]},

    # elasticnet penalty (Solver: saga) - Cuidado: saga pode falhar, max_iter alto é essencial
    {'model__penalty': ['elasticnet'], 'model__solver': ['saga'], 'model__C': loguniform(0.0001, 100.0), 'model__l1_ratio': stats_uniform(0, 1), 'model__max_iter': [10000]},

    # l2 penalty (Solvers: newton-cg, lbfgs, liblinear, sag, saga)
    {'model__penalty': ['l2'], 'model__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 'model__C': loguniform(0.0001, 100.0), 'model__max_iter': [10000]},

    # None penalty (Solvers: newton-cg, lbfgs, sag, saga)
    {'model__penalty': [None], 'model__solver': ['newton-cg', 'lbfgs', 'sag', 'saga'], 'model__max_iter': [10000]},
]


**VALIDAÇAO CRUZADA ANINHADA**

In [16]:
perform_nested_cv_corrected_accuracy(X_train, y_train, pipeline, inner_tscv, outer_tscv, param_distributions, scoring, n_iter, RANDOM_STATE)


--- Outer Split 1/5 ---
Outer Train Data Shape (Original): (335, 36)
Outer Test Data Shape (Original): (334, 36)
Starting inner Randomized Search (SMOTETomek in Pipeline)...


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(


Inner Search completed.
Inner CV best params (refit='accuracy'): {'model__C': np.float64(0.0008632008168602544), 'model__max_iter': 10000, 'model__penalty': 'l1', 'model__solver': 'liblinear'}
Inner CV best accuracy score: 0.9563636363636364
Evaluating best model on the outer test fold...
Outer fold Accuracy score: 0.8203592814371258
--------------------------------------------------

--- Outer Split 2/5 ---
Outer Train Data Shape (Original): (669, 36)
Outer Test Data Shape (Original): (334, 36)
Starting inner Randomized Search (SMOTETomek in Pipeline)...


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(


Inner Search completed.
Inner CV best params (refit='accuracy'): {'model__C': np.float64(0.0008632008168602544), 'model__max_iter': 10000, 'model__penalty': 'l1', 'model__solver': 'liblinear'}
Inner CV best accuracy score: 0.872072072072072
Evaluating best model on the outer test fold...
Outer fold Accuracy score: 0.9700598802395209
--------------------------------------------------

--- Outer Split 3/5 ---
Outer Train Data Shape (Original): (1003, 36)
Outer Test Data Shape (Original): (334, 36)
Starting inner Randomized Search (SMOTETomek in Pipeline)...


...

Inner Search completed.
Inner CV best params (refit='accuracy'): {'model__C': np.float64(0.0008632008168602544), 'model__max_iter': 10000, 'model__penalty': 'l1', 'model__solver': 'liblinear'}
Inner CV best accuracy score: 0.904191616766467
Evaluating best model on the outer test fold...
Outer fold Accuracy score: 0.9401197604790419
--------------------------------------------------

--- Outer Split 4/5 ---
Outer Train Data Shape (Original): (1337, 36)
Outer Test Data Shape (Original): (334, 36)
Starting inner Randomized Search (SMOTETomek in Pipeline)...


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(


Inner Search completed.
Inner CV best params (refit='accuracy'): {'model__C': np.float64(0.0008632008168602544), 'model__max_iter': 10000, 'model__penalty': 'l1', 'model__solver': 'liblinear'}
Inner CV best accuracy score: 0.9126126126126126
Evaluating best model on the outer test fold...
Outer fold Accuracy score: 0.907185628742515
--------------------------------------------------

--- Outer Split 5/5 ---
Outer Train Data Shape (Original): (1671, 36)
Outer Test Data Shape (Original): (334, 36)
Starting inner Randomized Search (SMOTETomek in Pipeline)...
Inner Search completed.
Inner CV best params (refit='accuracy'): {'model__C': np.float64(0.0008632008168602544), 'model__max_iter': 10000, 'model__penalty': 'l1', 'model__solver': 'liblinear'}
Inner CV best accuracy score: 0.9093525179856113
Evaluating best model on the outer test fold...
Outer fold Accuracy score: 0.9520958083832335
--------------------------------------------------

--- Resultados Finais da Validação Cruzada Aninhad

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(


([0.8203592814371258,
  0.9700598802395209,
  0.9401197604790419,
  0.907185628742515,
  0.9520958083832335],
 [{'model__C': np.float64(0.0008632008168602544),
   'model__max_iter': 10000,
   'model__penalty': 'l1',
   'model__solver': 'liblinear'},
  {'model__C': np.float64(0.0008632008168602544),
   'model__max_iter': 10000,
   'model__penalty': 'l1',
   'model__solver': 'liblinear'},
  {'model__C': np.float64(0.0008632008168602544),
   'model__max_iter': 10000,
   'model__penalty': 'l1',
   'model__solver': 'liblinear'},
  {'model__C': np.float64(0.0008632008168602544),
   'model__max_iter': 10000,
   'model__penalty': 'l1',
   'model__solver': 'liblinear'},
  {'model__C': np.float64(0.0008632008168602544),
   'model__max_iter': 10000,
   'model__penalty': 'l1',
   'model__solver': 'liblinear'}],
 [np.float64(0.9563636363636364),
  np.float64(0.872072072072072),
  np.float64(0.904191616766467),
  np.float64(0.9126126126126126),
  np.float64(0.9093525179856113)],
 np.float64(0.052939